<a href="https://www.kaggle.com/code/nadaaglan/four-shapes-computer-vision-task?scriptVersionId=180244962" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# Helper functions

- `imread`: Read an image from `img_path` and convert it to RGB
- `imshow`: Display the given `img` (`figsize` is optional figure size)
- `rgb2bin`: Convert the given RGB image `img_rgb` to binary
- `find_best_contour`: Find and return the largest contour in the image

In [ ]:
def imread(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def imshow(img, figsize=(5, 5)):
    plt.figure(figsize=figsize)
    plt.imshow(img, cmap='gray')
    plt.show()

def rgb2bin(img_rgb):
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    t, img_bin = cv2.threshold(
        img_gray, 0, 255, cv2.THRESH_OTSU|cv2.THRESH_BINARY_INV
    )
    return img_bin

def find_best_contour(img_rgb):
    img_bin = rgb2bin(img_rgb)
    contours, h = cv2.findContours(
        img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
    )
    best_contour = max(contours, key=cv2.contourArea)
    return best_contour

def draw_contours(img, contours, index=-1, color=(255, 0, 0), thickness=2):
    img_copy = img.copy()
    cv2.drawContours(img_copy, contours, index, color, thickness)
    imshow(img_copy)

In [ ]:
img = imread('/kaggle/input/imagesss/triangle-hollow.png')
imshow(img)

In [ ]:
best_contour = find_best_contour(img)
draw_contours(img, best_contour)

# Chain code histogram

We loop through the best contour points to generate chain code like the following image. We find the chain code by calculating the difference in x (`dx`) and difference in y (`dy`) between each point and the next point.

<img src="images/chain-codes-diagram.png" alt="chian codes diagram" width="400"/>


In [ ]:
best_contour

In [ ]:
best_contour.shape

In [ ]:
best_contour[0][0]

In [ ]:
x1,y1=best_contour[0][0]
x1,y1

In [ ]:
x2,y2=best_contour[1][0]
x2,y2

In [ ]:
x2-x1,y2-y1

In [ ]:
if x2-x1 == -1 and  y2-y1 == 1 :
    code = 1
    print(code)

In [ ]:
code

In [ ]:
d={
    (-1,1):1
}

In [ ]:
d[(x2-x1,y2-y1)]

We create a lookup table that maps between (`dx`, `dy`) and the chain code value according to the image above:
- Each key in the table is (`dx`, `dy`)
- The value is the chain code value

In [ ]:
lookup_table = {
    (1, 0): 0,
    (1, -1): 1,
    (0, -1): 2,
    (-1, -1): 3,
    (-1, 0): 4,
    (-1, 1): 5,
    (0, 1): 6,
    (1, 1): 7
}

We can generate a histogram of chain codes by counting how many times each value appears in the chain code. The next cell will loop through points of the best contour to print the chain code and calculate the histogram.
- First, we initialize the histogram `hist` to zeros. The histogram has 8 values that represent the number of occurrences of each chain code value (from 0 to 7)
- After that, we loop through the points of the best contour (except the last point)
- Inside the loop, we store the current point (index `i`) in `pt1` and the next point (index `i+1`) in `pt2`
- We calculate `dx` and `dy` by subtracting the x and y values between `pt2` and `pt1`
- We give (`dx`, `dy`) to the lookup table to obtain the chain code value (we can print it if we want)
- We increment the histogram at the index of the chain code value `hist[code] += 1`

Outside the loop, we normalize the histogram by dividing it by the total sum: `hist/hist.sum()`. The normalized histogram will remain the same even if the object's size increases

In [ ]:
import numpy as np

hist = np.zeros((8,))

In [ ]:
hist.shape

In [ ]:
import numpy as np

hist = np.zeros((8,))

for i in range(len(best_contour)-1):
    (x1, y1) = best_contour[i][0]
    (x2, y2) = best_contour[i+1][0]
    dx = x2 - x1
    dy = y2 - y1
    code = lookup_table[(dx, dy)]
    print(code, end='')
    hist[code] += 1

print("\n")
print(hist/hist.sum())

We wrap what we did so far in a function that takes an RGB image `img_rgb` and returns the normalized histogram of chain codes. We can use this histogram as a feature descriptor for the image

In [ ]:
def chain_hist(img_rgb):

    best_contour = find_best_contour(img_rgb)

    lookup_table = {
        (1, 0): 0,
        (1, -1): 1,
        (0, -1): 2,
        (-1, -1): 3,
        (-1, 0): 4,
        (-1, 1): 5,
        (0, 1): 6,
        (1, 1): 7
    }

    hist = np.zeros((8,))
    for i in range(len(best_contour)-1):
        pt1 = best_contour[i][0]
        pt2 = best_contour[i+1][0]
        dx = pt2[0] - pt1[0]
        dy = pt2[1] - pt1[1]
        code = lookup_table[(dx, dy)]
        hist[code] += 1

    return hist/hist.sum()

In [ ]:
print(chain_hist(img))

# Using Support Vector Machine for shape classification
We use the normalized histograms of chain codes as features to train a Support Vector machine (SVM) classifier to classify the images of shapes (circle, square, star, rectangle)

We use the Four shapes dataset from kaggle [dataset link](https://www.kaggle.com/datasets/smeschke/four-shapes)

The next function takes the path to the data folder (`data_path`) and returns a DataFrame with two columns: image path, and output

## Creating a DataFrame for our dataset

In [ ]:
import os
import pandas as pd

def create_df(data_path):

    # the class_dict is a mapping between class name and value
    class_dict = {
        'circle': 0,
        'square': 1,
        'star': 2,
        'triangle': 3
    }

    # we store image paths and outputs here
    df = []

    # for each class
    for class_name, class_value in class_dict.items():
        class_folder = os.path.join(data_path, class_name)
        # for each image in class folder
        for f in os.listdir(class_folder):
            f_path = os.path.join(class_folder, f)
            # if this is a "png" file, add its path and output
            if f_path.lower().endswith('.png'):
                df.append([f_path, class_value])

    # create a dataframe of image paths and outputs
    df = pd.DataFrame(df, columns=['path', 'output'])

    return df

We use the previous function to read the data from the `shapes_dataset/` folder

In [ ]:
data = create_df('/kaggle/input/four-shapes/shapes')
data

## Extracting features

We loop through image paths to read each image and extract features from it using our `chain_hist` function. We store all images' features in an array `X` and return it

In [ ]:
from tqdm import tqdm

def extract_features(img_paths):
    n = len(img_paths)
    X = np.zeros((n, 8))
    for i in tqdm(range(n)):
        f_path = img_paths[i]
        img = imread(f_path)
        features = chain_hist(img)
        X[i] = features
    return X

In [ ]:
X = extract_features(data['path'])

We store outputs in `y`

In [ ]:
y = data['output']

## Splitting dataset

We split the data to train and test sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=0
)

## Training

We train our Support vector machine

In [ ]:
from sklearn.svm import SVC

In [ ]:
model = SVC()
model.fit(X_train, y_train)

## Calculating accuracy

We print the accuracy of train and test

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print(accuracy_score(y_train, y_pred_train))
print(accuracy_score(y_test, y_pred_test))

## Testing with images outside the dataset

We test the model with images outside the dataset

In [ ]:
def test_model(model, img_path):
    classes = ['circle', 'square', 'star', 'triangle']
    img = imread(img_path)
    x = chain_hist(img)
    c = model.predict([x])[0]
    print(img_path, '->', classes[c])

In [ ]:
test_model(model, '/kaggle/input/imagesss/Square.png')
test_model(model, '/kaggle/input/imagesss/star.png')
test_model(model, '/kaggle/input/imagesss/circle.png')
test_model(model, '/kaggle/input/imagesss/triangle.png')
test_model(model, '/kaggle/input/imagesss/square-hollow.png')
test_model(model, '/kaggle/input/imagesss/star-hollow.png')
test_model(model, '/kaggle/input/imagesss/circle-hollow.png')
test_model(model, '/kaggle/input/imagesss/triangle-hollow.png')

We can see the model is correct for all images except the circle.

This problem can be solved by using a better dataset for training (The contour of the circles in the dataset are not smooth like the circles outside the dataset).

Or we can use a better feature descriptor for the images instead of the histogram of chain codes.

# Object detection example

In [ ]:
img = imread('/kaggle/input/imagesss/shapes.png')
imshow(img)

In [ ]:
def test_model(model, img_path):
    classes = ['circle', 'square', 'star', 'triangle']
    img = imread(img_path)
    img_copy = img.copy()  # Create a copy to draw on
    img_rgb = cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB)  # Convert to RGB for consistency
    
    # Find contours
    contours, _ = cv2.findContours(rgb2bin(img_rgb), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        # Calculate bounding rectangle
        x, y, w, h = cv2.boundingRect(contour)
        # bounding rectangle
        cv2.rectangle(img_copy, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        # Get the centroid of the bounding rectangle
        centroid_x = x + w //10
        centroid_y = y + h // 100
        
        # Get the predicted class
        x = chain_hist(img[y:y+h, x:x+w])  # Extract features from the bounding box region
        c = model.predict([x])[0]
        class_name = classes[c]
        
        # Write class name on top of the bounding box
        cv2.putText(img_copy, class_name, (centroid_x, centroid_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
    
    # Display the image with bounding boxes and labels
    imshow(img_copy)


In [ ]:
# Assuming you have already defined the `test_model` function

# Provide the file path of the image you want to test
image_path = "/kaggle/input/imagesss/shapes.png"

# Call the test_model function with the image path
test_model(model, image_path)


Try to write a program to take the previous image as input and produce the following output:

<img src="images/shapes-result-example.png" width="400" align="left">

# Other feature descriptors
The method we used is very simple and won't work for complex objects (such as cats). If we want to deal with complex objects, we should use a better feature descriptor. Examples include:
- SIFT (Scale-Invariant Feature Transform)
- SURF (Speeded-Up Robust Features)
- LBP (Local Binary Patterns)
- HOG (Histogram Oriented Gradients)
- Deep-learning based:
  - Auto encoders
  - Pre-trained models such as ResNet, etc.